<a href="https://colab.research.google.com/github/kyle-gao/ML_ipynb/blob/master/TF_TPU_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.


In [44]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Playing around with the TPU tutorial notebook before training transformer on TPU


In [45]:
import tensorflow as tf
import os
import tensorflow_datasets as tfds
import time

In [46]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.114.92.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.114.92.50:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU')]


In [47]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [48]:
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)])

In [53]:
def get_dataset(batch_size, is_training=True):
  split = 'train' if is_training else 'test'
  dataset, info = tfds.load(name='mnist', split=split, with_info=True,
                            as_supervised=True, try_gcs=True)

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0

    return image, label

  dataset = dataset.map(scale)

  # Only shuffle and repeat the dataset in training. The advantage to have a
  # infinite dataset for training is to avoid the potential last partial batch
  # in each epoch, so users don't need to think about scaling the gradients
  # based on the actual batch size.
  if is_training:
    dataset = dataset.shuffle(10000)
    dataset = dataset.repeat()

  #dataset = dataset.batch(batch_size)
  dataset = dataset.batch(batch_size).prefetch(8)

  return dataset

In [54]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

batch_size = 200
steps_per_epoch = 60000 // batch_size
validation_steps = 10000 // batch_size

train_dataset = get_dataset(batch_size, is_training=True)
test_dataset = get_dataset(batch_size, is_training=False)

model.fit(train_dataset,
          epochs=5,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset, 
          validation_steps=validation_steps,verbose = 2)

Epoch 1/5


300/300 - 15s - loss: 0.1363 - sparse_categorical_accuracy: 0.9572 - val_loss: 0.0571 - val_sparse_categorical_accuracy: 0.9830
Epoch 2/5
300/300 - 9s - loss: 0.0334 - sparse_categorical_accuracy: 0.9895 - val_loss: 0.0364 - val_sparse_categorical_accuracy: 0.9886
Epoch 3/5
300/300 - 9s - loss: 0.0206 - sparse_categorical_accuracy: 0.9934 - val_loss: 0.0453 - val_sparse_categorical_accuracy: 0.9885
Epoch 4/5
300/300 - 9s - loss: 0.0124 - sparse_categorical_accuracy: 0.9961 - val_loss: 0.0450 - val_sparse_categorical_accuracy: 0.9888
Epoch 5/5
300/300 - 9s - loss: 0.0103 - sparse_categorical_accuracy: 0.9967 - val_loss: 0.0504 - val_sparse_categorical_accuracy: 0.9880


In [ ]:
#TPU 11s/epoch
#no cache
#no pretech

"""Epoch 1/5
WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/multi_device_iterator_ops.py:601: get_next_as_optional (from tensorflow.python.data.ops.iterator_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/multi_device_iterator_ops.py:601: get_next_as_optional (from tensorflow.python.data.ops.iterator_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
  1/300 [..............................] - ETA: 22:40 - loss: 2.3021 - sparse_categorical_accuracy: 0.1300WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0026s vs `on_train_batch_end` time: 0.0308s). Check your callbacks.
WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0026s vs `on_train_batch_end` time: 0.0308s). Check your callbacks.
299/300 [============================>.] - ETA: 0s - loss: 0.1350 - sparse_categorical_accuracy: 0.9583WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0014s vs `on_test_batch_end` time: 0.0115s). Check your callbacks.
WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0014s vs `on_test_batch_end` time: 0.0115s). Check your callbacks.
300/300 [==============================] - 18s 60ms/step - loss: 0.1347 - sparse_categorical_accuracy: 0.9585 - val_loss: 0.0443 - val_sparse_categorical_accuracy: 0.9863
Epoch 2/5
300/300 [==============================] - 11s 36ms/step - loss: 0.0337 - sparse_categorical_accuracy: 0.9896 - val_loss: 0.0404 - val_sparse_categorical_accuracy: 0.9881
Epoch 3/5
300/300 [==============================] - 11s 36ms/step - loss: 0.0189 - sparse_categorical_accuracy: 0.9938 - val_loss: 0.0465 - val_sparse_categorical_accuracy: 0.9871
Epoch 4/5
300/300 [==============================] - 11s 37ms/step - loss: 0.0122 - sparse_categorical_accuracy: 0.9959 - val_loss: 0.0542 - val_sparse_categorical_accuracy: 0.9859
Epoch 5/5
300/300 [==============================] - 11s 36ms/step - loss: 0.0109 - sparse_categorical_accuracy: 0.9964 - val_loss: 0.0484 - val_sparse_categorical_accuracy: 0.9869
<tensorflow.python.keras.callbacks.History at 0x7fcf79ed3400>"""

#TPU 11s/epoch
#with cache
#with pretech

"""
Epoch 1/5
  1/300 [..............................] - ETA: 15:57 - loss: 2.3030 - sparse_categorical_accuracy: 0.1100WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0025s vs `on_train_batch_end` time: 0.0316s). Check your callbacks.
WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0025s vs `on_train_batch_end` time: 0.0316s). Check your callbacks.
299/300 [============================>.] - ETA: 0s - loss: 0.1310 - sparse_categorical_accuracy: 0.9584WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0015s vs `on_test_batch_end` time: 0.0129s). Check your callbacks.
WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0015s vs `on_test_batch_end` time: 0.0129s). Check your callbacks.
300/300 [==============================] - 16s 54ms/step - loss: 0.1306 - sparse_categorical_accuracy: 0.9585 - val_loss: 0.0464 - val_sparse_categorical_accuracy: 0.9853
Epoch 2/5
300/300 [==============================] - 11s 36ms/step - loss: 0.0342 - sparse_categorical_accuracy: 0.9891 - val_loss: 0.0398 - val_sparse_categorical_accuracy: 0.9874
Epoch 3/5
300/300 [==============================] - 11s 36ms/step - loss: 0.0188 - sparse_categorical_accuracy: 0.9940 - val_loss: 0.0415 - val_sparse_categorical_accuracy: 0.9869
Epoch 4/5
300/300 [==============================] - 11s 36ms/step - loss: 0.0131 - sparse_categorical_accuracy: 0.9958 - val_loss: 0.0455 - val_sparse_categorical_accuracy: 0.9864
Epoch 5/5
300/300 [==============================] - 11s 37ms/step - loss: 0.0087 - sparse_categorical_accuracy: 0.9970 - val_loss: 0.0475 - val_sparse_categorical_accuracy: 0.9876
<tensorflow.python.keras.callbacks.History at 0x7fcf79fbe320>"""

#Something is bottlenecking each batch. 
#TPU 9s/epoch
#with cache
#with pretech
#verbose = 2

"""Epoch 1/5
WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0019s vs `on_train_batch_end` time: 0.0290s). Check your callbacks.
WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0019s vs `on_train_batch_end` time: 0.0290s). Check your callbacks.
WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0014s vs `on_test_batch_end` time: 0.0116s). Check your callbacks.
WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0014s vs `on_test_batch_end` time: 0.0116s). Check your callbacks.
300/300 - 15s - loss: 0.1381 - sparse_categorical_accuracy: 0.9587 - val_loss: 0.0423 - val_sparse_categorical_accuracy: 0.9851
Epoch 2/5
300/300 - 9s - loss: 0.0339 - sparse_categorical_accuracy: 0.9891 - val_loss: 0.0395 - val_sparse_categorical_accuracy: 0.9877
Epoch 3/5
300/300 - 9s - loss: 0.0190 - sparse_categorical_accuracy: 0.9938 - val_loss: 0.0381 - val_sparse_categorical_accuracy: 0.9881
Epoch 4/5
300/300 - 9s - loss: 0.0113 - sparse_categorical_accuracy: 0.9959 - val_loss: 0.0455 - val_sparse_categorical_accuracy: 0.9877
Epoch 5/5
300/300 - 10s - loss: 0.0101 - sparse_categorical_accuracy: 0.9967 - val_loss: 0.0435 - val_sparse_categorical_accuracy: 0.9890
<tensorflow.python.keras.callbacks.History at 0x7fcf6f98bc88>"""


In [55]:
# Create the model, optimizer and metrics inside strategy scope, so that the
# variables can be mirrored on each device.
with strategy.scope():
  model = create_model()
  optimizer = tf.keras.optimizers.Adam()
  training_loss = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)
  training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      'training_accuracy', dtype=tf.float32)

# Calculate per replica batch size, and distribute the datasets on each TPU
# worker.
per_replica_batch_size = batch_size // strategy.num_replicas_in_sync

train_dataset = strategy.experimental_distribute_datasets_from_function(
    lambda _: get_dataset(per_replica_batch_size, is_training=True))

@tf.function
def train_step(iterator):
  """The step function for one training step"""

  def step_fn(inputs):
    """The computation to run on each TPU device."""
    images, labels = inputs
    with tf.GradientTape() as tape:
      logits = model(images, training=True)
      loss = tf.keras.losses.sparse_categorical_crossentropy(
          labels, logits, from_logits=True)
      loss = tf.nn.compute_average_loss(loss, global_batch_size=batch_size)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
    training_loss.update_state(loss * strategy.num_replicas_in_sync)
    training_accuracy.update_state(labels, logits)

  strategy.run(step_fn, args=(next(iterator),))

In [56]:
steps_per_eval = 10000 // batch_size

train_iterator = iter(train_dataset)
for epoch in range(5):
  tstart = time.time()
  print('Epoch: {}/5'.format(epoch))

  for step in range(steps_per_epoch):
    train_step(train_iterator)
  print('Current step: {}, training loss: {}, accuracy: {}%'.format(
      optimizer.iterations.numpy(),
      round(float(training_loss.result()), 4),
      round(float(training_accuracy.result()) * 100, 2)))
  tend = time.time()
  print('Time per epoch: {}/5'.format(tend-tstart))
  training_loss.reset_states()
  training_accuracy.reset_states()

Epoch: 0/5
Current step: 300, training loss: 0.1368, accuracy: 95.67%
Time per epoch: 12.046007871627808/5
Epoch: 1/5
Current step: 600, training loss: 0.0339, accuracy: 98.94%
Time per epoch: 6.1590354442596436/5
Epoch: 2/5
Current step: 900, training loss: 0.0205, accuracy: 99.36%
Time per epoch: 6.254938125610352/5
Epoch: 3/5
Current step: 1200, training loss: 0.0134, accuracy: 99.55%
Time per epoch: 6.231075048446655/5
Epoch: 4/5
Current step: 1500, training loss: 0.0113, accuracy: 99.63%
Time per epoch: 6.241615056991577/5


In [58]:
@tf.function
def train_multiple_steps(iterator, steps):
  
  """The step function for one training step"""

  def step_fn(inputs):
    """The computation to run on each TPU device."""
    images, labels = inputs
    with tf.GradientTape() as tape:
      logits = model(images, training=True)
      loss = tf.keras.losses.sparse_categorical_crossentropy(
          labels, logits, from_logits=True)
      loss = tf.nn.compute_average_loss(loss, global_batch_size=batch_size)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
    training_loss.update_state(loss * strategy.num_replicas_in_sync)
    training_accuracy.update_state(labels, logits)

  for _ in tf.range(steps):
    strategy.run(step_fn, args=(next(iterator),))

# Convert `steps_per_epoch` to `tf.Tensor` so the `tf.function` won't get 
# retraced if the value changes.
train_multiple_steps(train_iterator, tf.convert_to_tensor(steps_per_epoch))

print('Current step: {}, training loss: {}, accuracy: {}%'.format(
      optimizer.iterations.numpy(),
      round(float(training_loss.result()), 4),
      round(float(training_accuracy.result()) * 100, 2)))

Current step: 3300, training loss: 0.0059, accuracy: 99.81%
